# 简单项目2：客户购买预测

## 项目描述

使用逻辑回归预测客户是否会购买产品。这是一个典型的二分类问题，展示逻辑回归在业务场景中的应用。

## 学习目标

通过本项目，你将学会：
1. 如何处理多特征数据
2. 如何使用逻辑回归进行预测
3. 如何评估分类模型性能
4. 如何分析特征重要性

## 项目流程

1. 数据准备：生成或加载客户数据
2. 数据预处理：标准化特征
3. 模型训练：训练逻辑回归模型
4. 模型评估：评估模型性能
5. 特征分析：分析哪些特征最重要


In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report, 
                            confusion_matrix, roc_auc_score, roc_curve)
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
%matplotlib inline

print("环境准备完成！")


## 1. 数据准备

生成模拟客户数据，包括年龄、收入、浏览时间、点击次数、页面浏览量等特征。


In [ ]:
# 生成模拟客户数据
print("生成数据...")
X, y = make_classification(
    n_samples=1000,
    n_features=5,
    n_informative=3,  # 3个信息特征
    n_redundant=1,    # 1个冗余特征
    n_clusters_per_class=1,
    random_state=42
)

# 创建DataFrame，便于理解和分析
df = pd.DataFrame(X, columns=['age', 'income', 'browse_time', 
                               'click_count', 'page_views'])
df['purchase'] = y

print(f"数据形状: {df.shape}")
print(f"购买率: {df['purchase'].mean():.2%}")
print(f"\n数据预览:")
print(df.head())
print(f"\n数据统计:")
print(df.describe())


## 2. 数据预处理

划分训练集和测试集，并标准化特征。


In [ ]:
# 数据预处理
X = df.drop('purchase', axis=1)
y = df['purchase']

# 划分训练集和测试集（使用stratify保持类别分布）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 标准化特征（逻辑回归通常需要标准化）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"训练集购买率: {y_train.mean():.2%}")
print(f"测试集购买率: {y_test.mean():.2%}")


## 3. 模型训练

训练逻辑回归模型。


In [ ]:
# 训练模型
print("训练模型...")
model = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')
model.fit(X_train_scaled, y_train)
print("模型训练完成！")

# 查看模型参数
print(f"\n模型参数:")
print(f"偏置项: {model.intercept_[0]:.4f}")
print(f"\n特征系数:")
for i, col in enumerate(X.columns):
    print(f"  {col}: {model.coef_[0][i]:.4f}")


## 4. 模型评估

评估模型在测试集上的性能。


In [ ]:
# 预测
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# 评估指标
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print(f"准确率: {accuracy:.4f}")
print(f"ROC AUC: {auc:.4f}")

# 分类报告
print("\n分类报告:")
print(classification_report(y_test, y_pred, 
                            target_names=['未购买', '购买']))

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)
print("\n混淆矩阵:")
print(cm)

# ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, y_proba)


## 5. 结果可视化

可视化混淆矩阵、ROC曲线和特征重要性。


In [ ]:
# 可视化结果
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. 混淆矩阵
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['未购买', '购买'],
            yticklabels=['未购买', '购买'], ax=axes[0])
axes[0].set_title('混淆矩阵')
axes[0].set_ylabel('真实标签')
axes[0].set_xlabel('预测标签')

# 2. ROC曲线
axes[1].plot(fpr, tpr, label=f'ROC曲线 (AUC = {auc:.2f})', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', label='随机分类器')
axes[1].set_xlabel('假正例率 (FPR)')
axes[1].set_ylabel('真正例率 (TPR)')
axes[1].set_title('ROC曲线')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 3. 特征重要性（系数绝对值）
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'coefficient': model.coef_[0],
    'abs_coefficient': np.abs(model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

axes[2].barh(range(len(feature_importance)), feature_importance['abs_coefficient'])
axes[2].set_yticks(range(len(feature_importance)))
axes[2].set_yticklabels(feature_importance['feature'])
axes[2].set_xlabel('系数绝对值')
axes[2].set_title('特征重要性')
axes[2].invert_yaxis()

plt.tight_layout()
plt.show()

# 显示特征重要性
print("\n特征重要性（按系数绝对值排序）:")
print(feature_importance[['feature', 'coefficient', 'abs_coefficient']])


## 6. 模型应用示例

使用训练好的模型预测新客户是否会购买。


In [ ]:
# 模拟新客户数据
new_customers = pd.DataFrame({
    'age': [25, 35, 45],
    'income': [50, 80, 120],
    'browse_time': [10, 20, 30],
    'click_count': [5, 15, 25],
    'page_views': [8, 18, 28]
})

# 标准化新客户数据
new_customers_scaled = scaler.transform(new_customers)

# 预测
predictions = model.predict(new_customers_scaled)
probabilities = model.predict_proba(new_customers_scaled)[:, 1]

# 显示结果
print("新客户购买预测:")
print("-" * 50)
for i in range(len(new_customers)):
    print(f"\n客户 {i+1}:")
    print(f"  特征: {dict(new_customers.iloc[i])}")
    print(f"  预测: {'会购买' if predictions[i] == 1 else '不会购买'}")
    print(f"  购买概率: {probabilities[i]:.2%}")


## 7. 总结

### 项目总结

通过本项目，我们：
1. ✅ 学会了如何处理多特征数据
2. ✅ 使用逻辑回归进行客户购买预测
3. ✅ 评估了模型性能（准确率、ROC AUC、混淆矩阵）
4. ✅ 分析了特征重要性

### 改进方向

1. **特征工程**：创建新特征（如特征交互）
2. **模型优化**：调整正则化参数，使用交叉验证
3. **处理不平衡**：如果类别不平衡，使用class_weight或SMOTE
4. **特征选择**：使用L1正则化进行特征选择

### 思考问题

1. 哪些特征对购买预测最重要？为什么？
2. 如何解释逻辑回归的系数？
3. 如何提高模型的预测性能？
